In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

## Facebook Comment Volume Dataset

In [2]:
df = pd.read_csv('datasets/facebook_comments.csv', index_col=0)

In [3]:
print('dataset size: {}'.format(df.shape))

dataset size: (40949, 54)


In [4]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [5]:
y = df.Target.values.reshape(-1, 1)
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [6]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (40949, 228)


In [7]:
# number of components is fixed to 10
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

In [8]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [9]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data:
linear, MSE: 811.9202676506831
forest, MSE: 670.0735629931544


In [10]:
X, y = shuffle(X_pca, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [11]:
print('for decorrelated data using PCA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA
linear, MSE: 1260.2224961588795
forest, MSE: 1264.8861840248712


## Parkinsons Telemonitoring Data Set

In [12]:
df = pd.read_csv('datasets/parkinsons_updrs.data')

In [13]:
print('dataset size: {}'.format(df.shape))

dataset size: (5875, 22)


In [14]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [15]:
# replacing categorical features with binary values
y = df.total_UPDRS
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [16]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (5875, 61)


In [17]:
# number of components is fixed to 10
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

In [18]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [19]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data:
linear, MSE: 115.99671655195569
forest, MSE: 115.19258388486004


In [20]:
X, y = shuffle(X_pca, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [21]:
print('for decorrelated data using PCA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA
linear, MSE: 114.62534216524111
forest, MSE: 115.20916258526029


## Energy efficiency Data Set

In [22]:
df = pd.read_excel('./datasets/ENB2012_data.xlsx')

In [23]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [24]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [25]:
# replacing categorical features with binary values
y = df.Y1
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [26]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


In [27]:
# number of components is fixed to 10
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

In [28]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [29]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data:
linear, MSE: 103.98034793363026
forest, MSE: 106.44076601487366


In [30]:
X, y = shuffle(X_pca, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [31]:
print('for decorrelated data using PCA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA
linear, MSE: 103.0737275612831
forest, MSE: 108.9438503994889
